In [1]:
import os

In [2]:
%pwd

'/Users/kanayojustice/Documents/Data_scientist_projects/AutoPredict/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/kanayojustice/Documents/Data_scientist_projects/AutoPredict'

In [5]:
import os
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    target_column: str

In [7]:
from Autopredictor.src.constants import *
from Autopredictor.src.utils.common import read_yaml, create_directories


In [8]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config['artifacts_root']])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config_data = self.config['model_evaluation']
        create_directories([config_data['root_dir']])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=Path(config_data['root_dir']),
            test_data_path=Path(config_data['test_data_path']),
            model_path=Path(config_data['model_path']),
            metric_file_name=Path(config_data['metric_file_name']),
            target_column=self.schema['TARGET_COLUMN']['name']
        )

        return model_evaluation_config


In [9]:
import pandas as pd
from sklearn.metrics import (
    confusion_matrix, accuracy_score, precision_score, recall_score, 
    f1_score, roc_auc_score
)
from Autopredictor.src.utils.common import save_json
import joblib
from pathlib import Path
import logging

In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred, pred_proba):
        conf_matrix = confusion_matrix(actual, pred)
        accuracy = accuracy_score(actual, pred)
        precision = precision_score(actual, pred, pos_label='yes')  # Specify the positive label
        recall = recall_score(actual, pred, pos_label='yes')  # Specify the positive label
        f1 = f1_score(actual, pred, pos_label='yes')  # Specify the positive label
        roc_auc = roc_auc_score(actual, pred_proba)
        return conf_matrix, accuracy, precision, recall, f1, roc_auc

    def save_results(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]
        
        predicted_labels = model.predict(test_x)
        predicted_probabilities = model.predict_proba(test_x)[:, 1]

        (conf_matrix, accuracy, precision, recall, f1, roc_auc) = self.eval_metrics(test_y, predicted_labels, predicted_probabilities)
        
        # Saving metrics as local
        scores = {
            "confusion_matrix": conf_matrix.tolist(),
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
            "roc_auc": roc_auc
        }
        save_json(path=Path(self.config.metric_file_name), data=scores)


In [11]:
try:
        config_manager = ConfigurationManager()
        model_evaluation_config = config_manager.get_model_evaluation_config()

        model_evaluation = ModelEvaluation(config=model_evaluation_config)
        model_evaluation.save_results()
except Exception as e:
        logging.exception(e)
        raise e

[2024-05-30 15:08:01,433: INFO:common: yaml file: /Users/kanayojustice/Documents/Data_scientist_projects/AutoPredict/config/config.yaml loaded successfully]
[2024-05-30 15:08:01,438: INFO:common: yaml file: /Users/kanayojustice/Documents/Data_scientist_projects/AutoPredict/params.yaml loaded successfully]
[2024-05-30 15:08:01,442: INFO:common: yaml file: /Users/kanayojustice/Documents/Data_scientist_projects/AutoPredict/schema.yaml loaded successfully]
[2024-05-30 15:08:01,444: INFO:common: created directory at: artifacts]
[2024-05-30 15:08:01,446: INFO:common: created directory at: artifacts/model_evaluation]


[2024-05-30 15:08:02,947: INFO:common: json file saved at: artifacts/model_evaluation/metrics.json]
